In [2]:
# Code to implement VAE-gumple_softmax in pytorch
# author: Devinder Kumar (devinder.kumar@uwaterloo.ca)
# The code has been modified from pytorch example vae code 
# and inspired by the origianl tensorflow implementation of gumble-softmax by Eric Jang.

In [4]:
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [ ]:
parser = argparse.ArgumentParser(description='VAE MNIST Example')

parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--temp', type=float, default=1.0, metavar='S',
                    help='tau(temperature) (default: 1.0)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')



args = parser.parse_args()

In [7]:
print('VAE MNIST Example')
class Args():
    batch_size = 128
    epochs = 10
    temp = 1.0
    no_cuda = False
    seed = 1
    log_interval = 10
    hard = False

VAE MNIST Example


In [34]:
args = Args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    device = torch.device("cuda:0")

In [23]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/MNIST', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)

In [3]:
def sample_gumbel(shape, eps=1e-20):
    U = torch.rand(shape)
    if args.cuda:
        U = U.cuda()
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)


def gumbel_softmax(logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)
    
    if not hard:
#         return y
        return y.view(-1, latent_dim * categorical_dim)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
#     return y_hard
    return y_hard.view(-1, latent_dim * categorical_dim)

In [56]:
class VAE_gumbel(nn.Module):
    def __init__(self, temp):
        super(VAE_gumbel, self).__init__()

        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, latent_dim * categorical_dim)

        self.fc4 = nn.Linear(latent_dim * categorical_dim, 256)
        self.fc5 = nn.Linear(256, 512)
        self.fc6 = nn.Linear(512, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        return self.relu(self.fc3(h2))

    def decode(self, z):
        h4 = self.relu(self.fc4(z))
        h5 = self.relu(self.fc5(h4))
        return self.sigmoid(self.fc6(h5))

    def forward(self, x, temp, hard):
        print("x", x.shape)
        q = self.encode(x.view(-1, 784))
        print("after encoding", q.shape)
        q_y = q.view(q.size(0), latent_dim, categorical_dim)
        print("after reshaping", q_y.shape)
        z = gumbel_softmax(q_y, temp, hard)
        print("after gumbel softmax", z.shape)
        return self.decode(z), F.softmax(q_y, dim=-1).reshape(*q.size())

In [14]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, qy):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False) / x.shape[0]

    log_ratio = torch.log(qy * categorical_dim + 1e-20)
    KLD = torch.sum(qy * log_ratio, dim=-1).mean()

    return BCE + KLD

In [57]:
latent_dim = 30
categorical_dim = 10  # one-of-K vector

temp_min = 0.5
ANNEAL_RATE = 0.00003

model = VAE_gumbel(args.temp)
if args.cuda:
    model.cuda(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [58]:
epoch = 1
model.train()
train_loss = 0
temp = args.temp

for batch_idx, (data, _) in enumerate(train_loader):
    if args.cuda:
        data = data.cuda(device)
    if batch_idx == 2:
        break

In [59]:
print(data.shape)

torch.Size([128, 1, 28, 28])


In [61]:
recon_batch, qy = model(data, temp, args.hard)

x torch.Size([128, 1, 28, 28])
after encoding torch.Size([128, 300])
after reshaping torch.Size([128, 30, 10])
after gumbel softmax torch.Size([128, 300])


In [44]:
print(recon_batch.shape)

torch.Size([128, 784])


In [48]:
optimizer.zero_grad()
loss = loss_function(recon_batch, data, qy)
loss.backward()
train_loss += loss.item() * len(data)
optimizer.step()
# if batch_idx % 100 == 1:
#     temp = np.maximum(temp * np.exp(-ANNEAL_RATE * batch_idx), temp_min)


print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
    epoch, batch_idx * len(data), len(train_loader.dataset),
           100. * batch_idx / len(train_loader),
           loss.item()))



Train Epoch: 1 [256/60000 (0%)]	Loss: 107.393188


/nv/hp22/ashanker9/data/anaconda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [37]:
def train(epoch):
    model.train()
    train_loss = 0
    temp = args.temp
    for batch_idx, (data, _) in enumerate(train_loader):
        if args.cuda:
            data = data.cuda(device)
        optimizer.zero_grad()
        recon_batch, qy = model(data, temp, args.hard)
        loss = loss_function(recon_batch, data, qy)
        loss.backward()
        train_loss += loss.item() * len(data)
        optimizer.step()
        if batch_idx % 100 == 1:
            temp = np.maximum(temp * np.exp(-ANNEAL_RATE * batch_idx), temp_min)

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item()))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [38]:
def test(epoch):
    model.eval()
    test_loss = 0
    temp = args.temp
    for i, (data, _) in enumerate(test_loader):
        if args.cuda:
            data = data.cuda(device)
        recon_batch, qy = model(data, temp, args.hard)
        test_loss += loss_function(recon_batch, data, qy).item() * len(data)
        if i % 100 == 1:
            temp = np.maximum(temp * np.exp(-ANNEAL_RATE * i), temp_min)
        if i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                    recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
            save_image(comparison.data.cpu(),
                       'data/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [39]:
def run():
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test(epoch)

        M = 64 * latent_dim
        np_y = np.zeros((M, categorical_dim), dtype=np.float32)
        np_y[range(M), np.random.choice(categorical_dim, M)] = 1
        np_y = np.reshape(np_y, [M // latent_dim, latent_dim, categorical_dim])
        sample = torch.from_numpy(np_y).view(M // latent_dim, latent_dim * categorical_dim)
        if args.cuda:
            sample = sample.cuda()
        sample = model.decode(sample).cpu()
        save_image(sample.data.view(M // latent_dim, 1, 28, 28),
                   'data/sample_' + str(epoch) + '.png')

In [40]:
run()

/nv/hp22/ashanker9/data/anaconda/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1 [0/60000 (0%)]	Loss: 537.652039
Train Epoch: 1 [1280/60000 (2%)]	Loss: 256.205170
Train Epoch: 1 [2560/60000 (4%)]	Loss: 221.186722
Train Epoch: 1 [3840/60000 (6%)]	Loss: 213.886459
Train Epoch: 1 [5120/60000 (9%)]	Loss: 205.704697
Train Epoch: 1 [6400/60000 (11%)]	Loss: 210.139740
Train Epoch: 1 [7680/60000 (13%)]	Loss: 205.863556
Train Epoch: 1 [8960/60000 (15%)]	Loss: 205.054214
Train Epoch: 1 [10240/60000 (17%)]	Loss: 205.187408
Train Epoch: 1 [11520/60000 (19%)]	Loss: 199.648132
Train Epoch: 1 [12800/60000 (21%)]	Loss: 208.172195
Train Epoch: 1 [14080/60000 (23%)]	Loss: 204.667023
Train Epoch: 1 [15360/60000 (26%)]	Loss: 198.270447
Train Epoch: 1 [16640/60000 (28%)]	Loss: 195.610199
Train Epoch: 1 [17920/60000 (30%)]	Loss: 200.066452
Train Epoch: 1 [19200/60000 (32%)]	Loss: 199.849731
Train Epoch: 1 [20480/60000 (34%)]	Loss: 200.714066
Train Epoch: 1 [21760/60000 (36%)]	Loss: 202.974152
Train Epoch: 1 [23040/60000 (38%)]	Loss: 200.301788
Train Epoch: 1 [24320/60000 

Train Epoch: 4 [19200/60000 (32%)]	Loss: 136.205048
Train Epoch: 4 [20480/60000 (34%)]	Loss: 129.863556
Train Epoch: 4 [21760/60000 (36%)]	Loss: 128.170197
Train Epoch: 4 [23040/60000 (38%)]	Loss: 135.692108
Train Epoch: 4 [24320/60000 (41%)]	Loss: 127.822273
Train Epoch: 4 [25600/60000 (43%)]	Loss: 136.651077
Train Epoch: 4 [26880/60000 (45%)]	Loss: 136.246841
Train Epoch: 4 [28160/60000 (47%)]	Loss: 130.591034
Train Epoch: 4 [29440/60000 (49%)]	Loss: 128.744598
Train Epoch: 4 [30720/60000 (51%)]	Loss: 134.469986
Train Epoch: 4 [32000/60000 (53%)]	Loss: 131.433334
Train Epoch: 4 [33280/60000 (55%)]	Loss: 130.122360
Train Epoch: 4 [34560/60000 (58%)]	Loss: 125.520088
Train Epoch: 4 [35840/60000 (60%)]	Loss: 138.029160
Train Epoch: 4 [37120/60000 (62%)]	Loss: 134.562454
Train Epoch: 4 [38400/60000 (64%)]	Loss: 120.790787
Train Epoch: 4 [39680/60000 (66%)]	Loss: 127.493309
Train Epoch: 4 [40960/60000 (68%)]	Loss: 133.040817
Train Epoch: 4 [42240/60000 (70%)]	Loss: 134.980988
Train Epoch:

Train Epoch: 7 [38400/60000 (64%)]	Loss: 119.634399
Train Epoch: 7 [39680/60000 (66%)]	Loss: 121.222160
Train Epoch: 7 [40960/60000 (68%)]	Loss: 121.616173
Train Epoch: 7 [42240/60000 (70%)]	Loss: 120.739792
Train Epoch: 7 [43520/60000 (72%)]	Loss: 123.769028
Train Epoch: 7 [44800/60000 (75%)]	Loss: 117.854111
Train Epoch: 7 [46080/60000 (77%)]	Loss: 115.417442
Train Epoch: 7 [47360/60000 (79%)]	Loss: 117.746674
Train Epoch: 7 [48640/60000 (81%)]	Loss: 113.261147
Train Epoch: 7 [49920/60000 (83%)]	Loss: 114.090500
Train Epoch: 7 [51200/60000 (85%)]	Loss: 117.125168
Train Epoch: 7 [52480/60000 (87%)]	Loss: 117.867554
Train Epoch: 7 [53760/60000 (90%)]	Loss: 118.667244
Train Epoch: 7 [55040/60000 (92%)]	Loss: 114.186890
Train Epoch: 7 [56320/60000 (94%)]	Loss: 121.170830
Train Epoch: 7 [57600/60000 (96%)]	Loss: 114.187851
Train Epoch: 7 [58880/60000 (98%)]	Loss: 121.132751
====> Epoch: 7 Average loss: 118.7791
====> Test set loss: 116.5663
Train Epoch: 8 [0/60000 (0%)]	Loss: 116.237717
T

Train Epoch: 10 [56320/60000 (94%)]	Loss: 112.396675
Train Epoch: 10 [57600/60000 (96%)]	Loss: 113.082603
Train Epoch: 10 [58880/60000 (98%)]	Loss: 109.897362
====> Epoch: 10 Average loss: 112.9084
====> Test set loss: 112.1237
